# Salary Predictions Based on Job Descriptions

### Problem : Examine job postings with salaries in the past and predict salaries for new set of job postings

In [27]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from statistics import *

# my info 
__author__ = "Aesha Chauhan"
__email__ = "aesha.c30@gmail.com"

### Load the data 

#### Load the data into a Pandas dataframe and get an overview of the data

In [3]:
train_features_df = pd.read_csv('/Users/aesha/Desktop/DSDJ/Salary-Prediction/data/train_features.csv')
test_features_df = pd.read_csv('/Users/aesha/Desktop/DSDJ/Salary-Prediction/data/test_features.csv')
train_target_df = pd.read_csv('/Users/aesha/Desktop/DSDJ/Salary-Prediction/data/train_Salaries.csv')

In [13]:
train_features_df.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16


In [14]:
test_features_df.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
1,JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
2,JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9
3,JOB1362685407690,COMP21,MANAGER,HIGH_SCHOOL,NONE,OIL,14,96
4,JOB1362685407691,COMP36,JUNIOR,DOCTORAL,BIOLOGY,OIL,10,44


In [15]:
train_target_df.head()

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163


In [5]:
train_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [6]:
test_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [7]:
train_target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   jobId   1000000 non-null  object
 1   salary  1000000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.3+ MB


In [31]:
mean(train_target_df.salary)

116.061818

In [30]:
stdev(train_target_df.salary)

38.71793638113314

In [32]:
median(train_target_df.salary)

114.0

In [36]:
variance(train_target_df.salary)

1499.0785976134737

In [37]:
min(train_target_df.salary)

0

In [38]:
max(train_target_df.salary)

301

### Clean the data

#### Look for duplicate, invalid, or corrupt data and remove it

In [50]:
train_features_df.duplicated().sum()

0

In [40]:
train_target_df.duplicated().sum()

0

In [41]:
test_features_df.duplicated().sum()

0

In [42]:
invalid_data=train_target_df[train_target_df.salary <=0]
invalid_data

,jobId,salary
30559,JOB1362684438246,0
495984,JOB1362684903671,0
652076,JOB1362685059763,0
816129,JOB1362685223816,0
828156,JOB1362685235843,0


In [47]:
train_target_df=train_target_df.drop(train_target_df.index[[30559,495984,652076,816129,828156]])
train_target_df

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163
...,...,...
999995,JOB1362685407682,88
999996,JOB1362685407683,160
999997,JOB1362685407684,64
999998,JOB1362685407685,149


In [48]:
min(train_target_df.salary)

17

### Explore the data (EDA) 

#### Identify numerical and categorical columns

In [53]:
train_features_df.columns

Index(['jobId', 'companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis'],
      dtype='object')

In [54]:
numeric_cols = ['yearsExperience','milesFromMetropolis']
categorical_cols = ['jobId','companyId','jobType','degree','major','industry']

#### Summarize numerical and categorical columns separately

In [55]:
train_features_df.describe(include=[np.number])

,yearsExperience,milesFromMetropolis
count,1000000.000000,1000000.000000
mean,11.992386,49.529260
std,7.212391,28.877733
min,0.000000,0.000000
25%,6.000000,25.000000
50%,12.000000,50.000000
75%,18.000000,75.000000
max,24.000000,99.000000


In [60]:
train_features_df.describe(include=['O'])

,jobId,companyId,jobType,degree,major,industry
count,1000000,1000000,1000000,1000000,1000000,1000000
unique,1000000,63,8,5,9,7
top,JOB1362685163706,COMP39,SENIOR,HIGH_SCHOOL,NONE,WEB
freq,1,16193,125886,236976,532355,143206


In [66]:
train_target_df.describe(include=[np.number])

,salary
count,999990.000000
mean,116.062461
std,38.717191
min,17.000000
25%,88.000000
50%,114.000000
75%,141.000000
max,301.000000


In [67]:
train_target_df.describe(include=['O'])

,jobId
count,999990
unique,999990
top,JOB1362685163706
freq,1


#### Merge features and target into single df

In [70]:
# merge features and salaries on jobId
train_df = pd.merge(train_features_df,train_target_df,on='jobId')
train_df.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163


In [71]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999990 entries, 0 to 999989
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   jobId                999990 non-null  object
 1   companyId            999990 non-null  object
 2   jobType              999990 non-null  object
 3   degree               999990 non-null  object
 4   major                999990 non-null  object
 5   industry             999990 non-null  object
 6   yearsExperience      999990 non-null  int64 
 7   milesFromMetropolis  999990 non-null  int64 
 8   salary               999990 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 76.3+ MB


In [3]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features